# LIS With MobileViTs

## 1. Our MobileViTs

In [1]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.vec.gz
!gunzip cc.de.300.vec.gz
!pip install lightning torchmetrics einops av # gensim





--2024-12-16 15:16:49--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.33, 13.227.219.10, 13.227.219.70, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1278030050 (1.2G) [binary/octet-stream]
Saving to: 'cc.de.300.vec.gz'

cc.de.300.vec.gz    100%[===================>]   1.19G   189MB/s    in 6.5s    

2024-12-16 15:16:56 (187 MB/s) - 'cc.de.300.vec.gz' saved [1278030050/1278030050]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 21.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 45.6 MB/s eta 0:00:00:00:0100:01


In [31]:
from gensim.models import KeyedVectors
import numpy as np
import lightning as pl
import torch
import torch.nn as nn
import torch.functional as F
import wandb
from einops import rearrange
import torchvision
from torch.utils.data import DataLoader
import os
import gc
import numpy as np
from torchmetrics.text import WordErrorRate
from tqdm import tqdm
import gzip
import pickle
from lightning.pytorch.utilities.model_summary import summarize
from lightning.fabric.utilities import measure_flops
import pdb

In [3]:


# Percorso del file scaricato
embedding_file = "/kaggle/working/cc.de.300.vec"

# Carica gli embedding FastText pre-addestrati
fasttext_model = KeyedVectors.load_word2vec_format(embedding_file, binary=False)
special_words = ['<start>', '<end>', '<unk>']
special_embeddings=[np.full(300, 1),np.full(300, 2), np.full(300, 3)]
fasttext_model.add_vectors(special_words,special_embeddings )

In [ ]:
!ls /kaggle/working

In [137]:
# Optimize this impl:
# https://github.com/chinhsuanwu/mobilevit-pytorch/blob/master/mobilevit.py
import pdb


class Conv2DBlock(nn.Module):

    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride,
        padding,
        groups=1,
        bias=True,
        norm=True,
        activation=True,
        dropout=0.1,
    ):
        """__init__ Constructor for Conv2DBlock

        Parameters
        ----------
        in_channels : int
            Number of input channels
        out_channels : int
            Number of output channels
        kernel_size : int
            Size of the kernel
        stride : int
            Stride of the convolutional layer
        padding : int
            Padding of the convolutional layer
        groups : int
            Number of groups
        bias : bool
            Whether to use bias
        dropout : float
            Dropout rate
        """

        super(Conv2DBlock, self).__init__()

        self.block = nn.Sequential()

        self.block.add_module(
            "conv2d",
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                groups=groups,
                bias=bias,
            ),
        )

        if norm:
            self.block.add_module("norm", nn.BatchNorm2d(out_channels))

        if activation:
            self.block.add_module("activation", nn.SiLU())  # sigmoid(x) * x // SWISH

        if dropout:
            self.block.add_module("dropout", nn.Dropout(dropout))

        self.block = self.block

    def forward(self, x):
        return self.block(x)


class MobileBlockV2(nn.Module):

    def __init__(self, in_channels, out_channels, stride, expand_ratio):
        """__init__ Constructor for MobileBlockV2


        Parameters
        ----------
        in_channels : int
            Number of input channels
        out_channels : int
            Number of output channels
        stride : int
            Stride of the convolutional layer
        expand_ratio : int
            Expansion ratio of the block
        """

        super(MobileBlockV2, self).__init__()

        assert stride in [1, 2], "Stride must be either 1 or 2"

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride
        self.expand_ratio = expand_ratio
        self.hidden_dim = int(round(in_channels * expand_ratio))

        self.mbv2 = nn.Sequential()
        self.uses_inverse_residual = (
            self.in_channels == self.out_channels and self.stride == 1
        )

        if self.expand_ratio == 1:
            self.mbv2.add_module(
                "depthwise_3x3",
                Conv2DBlock(  # Depthwise Convolution
                    in_channels=self.hidden_dim,
                    out_channels=self.hidden_dim,
                    kernel_size=3,
                    stride=stride,
                    padding=1,
                    groups=self.hidden_dim,
                    bias=False,
                    norm=True,
                    activation=True,
                ),
            )
            self.mbv2.add_module(
                "pointwise-linear_1x1",
                Conv2DBlock(  # Pointwise-Linear Convolution
                    in_channels=self.hidden_dim,
                    out_channels=self.hidden_dim,
                    kernel_size=1,
                    stride=1,
                    padding=0,
                    groups=self.hidden_dim,
                    bias=False,
                    norm=True,
                    activation=False,
                ),
            )
        else:
            self.mbv2.add_module(
                "pointwise_1x1",
                Conv2DBlock(  # Pointwise-Linear Convolution
                    in_channels=in_channels,
                    out_channels=self.hidden_dim,
                    kernel_size=1,
                    stride=1,
                    padding=0,
                    groups=1,
                    bias=False,
                    norm=True,
                    activation=True,
                ),
            )
            self.mbv2.add_module(
                "depthwise_3x3",
                Conv2DBlock(  # Depthwise Convolution
                    in_channels=self.hidden_dim,
                    out_channels=self.hidden_dim,
                    kernel_size=3,
                    stride=stride,
                    padding=1,
                    groups=self.hidden_dim,
                    bias=False,
                    norm=True,
                    activation=True,
                ),
            )
            self.mbv2.add_module(
                "pointwise-linear_1x1",
                Conv2DBlock(  # Pointwise-Linear Convolution
                    in_channels=self.hidden_dim,
                    out_channels=self.out_channels,
                    kernel_size=1,
                    stride=1,
                    padding=0,
                    groups=1,
                    bias=False,
                    norm=True,
                    activation=False,
                ),
            )

        self.mbv2 = self.mbv2

    def forward(self, x):
        if self.uses_inverse_residual:
            return x + self.mbv2(x)
        else:
            return self.mbv2(x)


class MobileViTBlock(nn.Module):

    def __init__(
        self, hidden_dim, depth, channels, kernel_size, patch_size, mlp_dim, dropout=0.1
    ):

        super(MobileViTBlock, self).__init__()

        self.hidden_dim = hidden_dim
        self.depth = depth
        self.channels = channels
        self.kernel_size = kernel_size
        self.patch_size = patch_size
        self.mlp_dim = mlp_dim
        self.dropout = dropout

        self.local_conv = nn.Sequential(
            Conv2DBlock(
                in_channels=channels,
                out_channels=channels,
                kernel_size=kernel_size,
                stride=1,
                padding=1,
                norm=True,
                activation=True,
                bias=False,
            ),
            Conv2DBlock(
                in_channels=channels,
                out_channels=hidden_dim,
                kernel_size=1,
                stride=1,
                padding=0,
                norm=True,
                activation=True,
                bias=False,
            ),
        )

        self.global_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=hidden_dim,
                nhead=8,
                dim_feedforward=mlp_dim,
                dropout=dropout,
                batch_first=True,
                activation="gelu",
            ),
            num_layers=depth,
            norm=nn.LayerNorm(hidden_dim),
        )

        self.fusion_conv_preres = Conv2DBlock(
            in_channels=hidden_dim,
            out_channels=channels,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
            norm=True,
            activation=True,
        )

        self.fusion_conv_postres = Conv2DBlock(
            in_channels=2 * channels,
            out_channels=channels,
            kernel_size=kernel_size,
            stride=1,
            padding=1,
            bias=False,
            norm=True,
            activation=True,
        )

    def forward(self, x):

        B, T, C, H, W = x.shape
        x = rearrange(x, "b t c h w -> (b t) c h w")
        x_res = x.clone()

        # local_repr
        x = self.local_conv(x)

        ph, pw = self.patch_size

        # global_repr
        _, _, h, w = x.shape
        x = rearrange(  # reshape the image into patches for ViT input
            x,
            "(b t) d (h ph) (w pw) -> (b h w) (t ph pw) d",
            ph=ph,
            pw=pw,
            b=B,
            t=T,
        )
        x = self.global_transformer(x)
        x = rearrange(
            x,
            "(b h w) (t ph pw) d -> (b t) d (h ph) (w pw)",
            h=h // ph,
            w=w // pw,
            ph=ph,
            pw=pw,
            b=B,
            t=T,
        )

        # fusion
        x = self.fusion_conv_preres(x)
        x = torch.cat([x, x_res], dim=1)
        x = self.fusion_conv_postres(x)

        x = rearrange(x, "(b t) c h w -> b t c h w", b=B, t=T)

        return x


class VideoDecoder(nn.Module):
    def __init__(self, input_dim,hidden_dim, output_dim,vocab, num_layers=1, dropout=0.1, debug=False):
        super(VideoDecoder, self).__init__()
        self.debug=debug
        
        # RNN o GRU o Transformer Decoder, scegli tu l'architettura
        self.decoder = nn.TransformerDecoderLayer(
            d_model=input_dim,
            nhead=8,  # Numero di teste di attenzione, personalizzabile
            batch_first=True,
            dropout=dropout,
            activation='gelu'  # Funzione di attivazione, puoi cambiarla in base alle esigenze
        )
        self.vocab =  vocab
        self.embed=nn.Linear(300,320)
        self.output_layer = nn.Linear(input_dim, output_dim)
        self.mask = torch.tril(torch.ones(64,64))

    def pad_tensor(self, tensor, max_len, embed_dim=300, pad_value=-1):
        """
        Aggiunge padding lungo la dimensione della sequenza (seqLen) e crea un padding con una dimensione aggiuntiva.
        
        Args:
            tensor (torch.Tensor): Tensore originale di dimensioni (batch x seqLen x embedDim).
            max_len (int): Lunghezza massima desiderata per seqLen.
            embed_dim (int): Dimensione degli embedding (es. 300).
            pad_value (int): Valore da usare per il padding (default: -1).
        
        Returns:
            torch.Tensor: Tensore di dimensioni (batch x max_len x embedDim) con il padding aggiunto.
        """
        seq_len = tensor.size(0)
        # assert current_embed_dim == embed_dim, "La dimensione degli embedding non corrisponde a embed_dim."
    
        
        # Calcola la lunghezza del padding necessario
        padding_length = max_len - seq_len
        # Crea il padding con le dimensioni corrette
        padding = torch.full(
            (batch_size, padding_length, embed_dim), pad_value, dtype=tensor.dtype, device=tensor.device
        )
        # Concatena il tensore originale con il padding lungo la dimensione seqLen (dim=1)
        
        
        return padding


        
    def forward(self, tgt,padding_mask, memory):
        if self.debug:
            pdb.set_trace()
        tgt = self.embed(tgt)
        # encoder_output avrà dimensione (batch_size, seq_length, input_dim)
        output = self.decoder(tgt=tgt.to(dtype=torch.float32), memory=memory, tgt_mask=self.mask, \
                              tgt_key_padding_mask=padding_mask)
        output = self.output_layer(output)
        
        return output


class MobileViT(pl.LightningModule):

    def __init__(
        self,
        dims,
        conv_channels,
        num_classes,
        vocabulary,
        expand_ratio=4,
        patch_size=(2, 2),
        debug= False
    ):
        super(MobileViT, self).__init__()

        self.dims = dims
        self.conv_channels = conv_channels
        self.num_classes = num_classes
        self.expand_ratio = expand_ratio
        self.patch_size = patch_size
        self.kernel_size = 3
        self.debug = debug

        L = [2, 4, 3]

        self.in_conv = Conv2DBlock(
            in_channels=3,
            out_channels=conv_channels[0],
            kernel_size=self.kernel_size,
            stride=2,
            padding=1,
            norm=True,
            activation=True,
            bias=False,
        )

        self.mv2_blocks = nn.ModuleList([])

        self.mv2_blocks.append(
            MobileBlockV2(
                conv_channels[0], conv_channels[1], 1, expand_ratio=expand_ratio
            )
        )
        self.mv2_blocks.append(
            MobileBlockV2(
                conv_channels[1], conv_channels[2], 2, expand_ratio=expand_ratio
            )
        )
        self.mv2_blocks.append(
            MobileBlockV2(
                conv_channels[2], conv_channels[3], 1, expand_ratio=expand_ratio
            )
        )
        self.mv2_blocks.append(
            MobileBlockV2(
                conv_channels[2], conv_channels[3], 1, expand_ratio=expand_ratio
            )
        )
        self.mv2_blocks.append(
            MobileBlockV2(
                conv_channels[3], conv_channels[4], 2, expand_ratio=expand_ratio
            )
        )
        self.mv2_blocks.append(
            MobileBlockV2(
                conv_channels[5], conv_channels[6], 2, expand_ratio=expand_ratio
            )
        )
        self.mv2_blocks.append(
            MobileBlockV2(
                conv_channels[7], conv_channels[8], 2, expand_ratio=expand_ratio
            )
        )

        self.mvit_blocks = nn.ModuleList([])

        self.mvit_blocks.append(
            MobileViTBlock(
                dims[0],
                L[0],
                conv_channels[5],
                self.kernel_size,
                patch_size,
                int(dims[0] * 2),
            )
        )
        self.mvit_blocks.append(
            MobileViTBlock(
                dims[1],
                L[1],
                conv_channels[7],
                self.kernel_size,
                patch_size,
                int(dims[1] * 4),
            )
        )
        self.mvit_blocks.append(
            MobileViTBlock(
                dims[2],
                L[2],
                conv_channels[9],
                self.kernel_size,
                patch_size,
                int(dims[2] * 4),
            )
        )

        self.final_pw = Conv2DBlock(
            in_channels=conv_channels[-2],
            out_channels=conv_channels[-1],
            kernel_size=1,
            stride=1,
            padding=0,
            groups=1,
            bias=False,
            norm=True,
            activation=False,
        )

        self.decoder = VideoDecoder(input_dim=320, hidden_dim=conv_channels[-1], output_dim=len(vocabulary), vocab = vocabulary, debug=debug)

        

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(conv_channels[-1], num_classes, bias=False)

        ## Training-related members
        self.criterion = nn.CrossEntropyLoss(ignore_index=-1)
        self.wer = WordErrorRate()

        self.vocabulary = vocabulary

        self.apply(self.init_weights)  # Initialize weights

    def init_weights(self, m):

        if type(m) == nn.Conv2d:
            nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif type(m) == nn.BatchNorm2d:
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
        elif type(m) == nn.Linear:
            nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

    def add_padding(self, sequences, max_len):
        seq_len = sequences.size()[0]
        if seq_len < max_len:
            padding = max_len - seq_len
            # Pad along the first dimension (batch dimension)
            padded_sequences = torch.nn.functional.pad(
                sequences, 
                (0, padding),  # Padding lungo la seconda dimensione (seq_len)
                value=-1
            )
        
        else:
            padded_sequences = sequences
        
    
        return padded_sequences

    

    def forward(self, x, embeddings, padding_mask):
        if self.debug:
            pdb.set_trace()
        
    

        B, T, C, H, W = x.shape
        x = rearrange(x, "b t c h w -> (b t) c h w")

        x = self.in_conv(x)

        for i in range(5):
            x = self.mv2_blocks[i](x)

        x = rearrange(x, "(b t) c h w -> b t c h w", b=B, t=T)
        x = self.mvit_blocks[0](x)

        x = rearrange(x, "b t c h w -> (b t) c h w")
        x = self.mv2_blocks[5](x)

        x = rearrange(x, "(b t) c h w -> b t c h w", b=B, t=T)
        x = self.mvit_blocks[1](x)

        x = rearrange(x, "b t c h w -> (b t) c h w")
        x = self.mv2_blocks[6](x)

        x = rearrange(x, "(b t) c h w -> b t c h w", b=B, t=T)
        x = self.mvit_blocks[2](x)

        x = rearrange(x, "b t c h w -> (b t) c h w", b=B, t=T)
        
        x = self.final_pw(x)
        x = self.pool(x)
        x = x.flatten(1).unsqueeze(0)
        x = self.decoder(embeddings,padding_mask,x )
        
        return x

    def step(self, batch, phase):
        if self.debug:
            pdb.set_trace()
        x, y,embeddings, padding_mask = batch

        B, T, C, H, W = x.shape
        B, N = y.shape

        # assume padding is done in the dataloader
        input_lengths = torch.full((B,), T, dtype=torch.long)
        target_lengths = torch.full((B,), N, dtype=torch.long)

        logits = self(x, embeddings, padding_mask)
        if self.debug:
            pdb.set_trace()

        log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
        # sum one to y as 0 is the blank token in our representation

        # out_sentence = torch.argmax(log_probs, dim=-1)
        # Rimodellare logits per adattarli a nn.CrossEntropyLoss
        logits_reshaped = log_probs.view(-1, log_probs.size(-1))  # [batch_size * seq_len, num_classes]
        
        # Rimodellare target per adattarli a nn.CrossEntropyLoss
        target_reshaped = y.view(-1)  # [batch_size * seq_len]
        
        loss = self.criterion(logits_reshaped, target_reshaped)

        self.log(f"{phase}/loss", loss, prog_bar=True)

        if phase != "train":
            word_error_rate = self.calculate_wer(log_probs, y)
            self.log(f"{phase}/wer", word_error_rate, prog_bar=True)

        return loss

    def calculate_wer(self, log_probs, y):
        if self.debug:
            pdb.set_trace()
        pred_strings = self.string_from_logits(log_probs)
        target_strings = self.string_from_ground_truth(y)

        pred_strings = [" ".join(pred) for pred in pred_strings]
        target_strings = [" ".join(target) for target in target_strings]
        if self.debug:
            pdb.set_trace()

        return self.wer(pred_strings, target_strings)

    def string_from_logits(self, log_probs):
        
        decoded = self.ctc_decode(log_probs)

        return self.string_from_ground_truth(decoded)

    def ctc_decode(self, logits, blank=0):
        logits = torch.argmax(logits, dim=-1)  # Take the most probable class
        decoded = []
        if self.debug:
            pdb.set_trace()
        for seq in logits.T:  # Iterate over batch
            result = []
            prev_token = blank
            for token in seq:
                if token != prev_token and token != blank:
                    result.append(token.item())
                prev_token = token
            decoded.append(result)
        if self.debug:
            pdb.set_trace()
        return decoded

    def string_from_ground_truth(self, y):
        target_strings = []
        if self.debug:
            pdb.set_trace()
        for target in y:
            if isinstance(target, torch.Tensor):
                target = target.tolist()
            
            target_strings.append(self.vocabulary.decode_from_ids(target))
        return target_strings

    def training_step(self, batch, batch_idx):
        return self.step(batch, "train")

    def validation_step(self, batch, batch_idx):
        return self.step(batch, "val")

    def test_step(self, batch, batch_idx):
        return self.step(batch, "test")

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(), lr=2e-4, amsgrad=True, weight_decay=1e-4
        )
        scheduler = {
            "scheduler": torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
                optimizer, T_0=10, T_mult=2, eta_min=1e-6
            ),
            "interval": "step",
            "frequency": 32,
        }

        return [optimizer], [scheduler]

# 2. Our Datamodules

In [140]:
from dataclasses import dataclass
import random


@dataclass
class PhoenixFiles:
    train: str
    dev: str
    test: str


@dataclass
class PhoenixDataHyperparameters:
    num_workers: int
    batch_size: int
    sample_random: bool
    sample_uniform: bool
    sample_percentage: float
    transforms: torchvision.transforms.Compose


class PhoenixVocabulary:

    def __init__(self, filenames: PhoenixFiles):
        self.filenames = filenames
        
        embedding_file = "/kaggle/working/cc.de.300.vec"

        # Carica gli embedding FastText pre-addestrati
        print("loading embeddings")
        self.fasttext_model = KeyedVectors.load_word2vec_format(embedding_file, binary=False)
        special_words = ['<start>', '<end>', '<unk>']
        special_embeddings=[np.full(300, 1),np.full(300, 2), np.full(300, 3)]
        self.fasttext_model.add_vectors(special_words,special_embeddings )
        self.vocab = self.build_vocab()
        self.vocab_inversed = {v[0]: k for k, v in self.vocab.items()}

    def get_embedding(self, word):
        # Crea un embedding speciale per OOV utilizzando l'embedding di <unk>
        
        return self.fasttext_model[word] if word in self.fasttext_model else torch.tensor(self.fasttext_model['<unk>'])
    

    def build_vocab(self):
        vocab = {}

        files = [self.filenames.train, self.filenames.dev, self.filenames.test]

        for file in files:
            with gzip.open(file, "rb") as f:
                annotations = pickle.load(f)

            for ann in tqdm(
                random.sample(annotations, len(annotations)),
                desc=f"Extracting tokens from {os.path.basename(file)}",
            ):

                # random.sample shuffles the strings, this improves token distribution (hopefully improves training)
                for word in random.sample(
                    ann["gloss"].split(), len(ann["gloss"].split())
                ):
                    if word not in vocab:
                        vocab[word] = (len(vocab), self.get_embedding(word))

        vocab["<start>"] = (len(vocab),self.get_embedding("<start>"))
        vocab["<end>"] = (len(vocab),self.get_embedding("<end>"))

        return vocab

    def __len__(self):
        return len(self.vocab)

    def __getitem__(self, idx):
        return self.vocab[idx]

    def encode_as_ids(self, sentence):
        return [self.vocab[word] for word in sentence]

    def decode_from_ids(self, ids):
        return [self.vocab_inversed[tkn] if tkn != -1 for tkn in ids]


class Phoenix14TDataset(torch.utils.data.Dataset):

    def __init__(self, path: str, vocabulary: PhoenixVocabulary, hyperparameters):
        self.metadata_path = path
        self.base_dir = os.path.dirname(path)
        self.data = None
        self.vocab: PhoenixVocabulary = vocabulary
        self.data_hparams = hyperparameters
        self.transform = hyperparameters.transforms

        assert (
            self.data_hparams.sample_random != self.data_hparams.sample_uniform
        ), "Both random and uniform sampling cannot be enabled at the same time"

    def setup(self, stage):
        with gzip.open(self.metadata_path, "rb") as f:
            self.data = pickle.load(f)


        self.signers = []   
        self.video_names = [] 
        self.annotations = []  
        self.text = []  
        self.targets = []
        self.embeddings = []
        for d in self.data:
            self.signers.append(d["signer"])
            self.video_names.append(d["name"])
            self.annotations.append(d["gloss"])
            self.text.append(d["text"])
        max_len = 0  
        for ann in tqdm(
                self.annotations,
                desc=(
                    f"Encoding annotations for {stage}"
                    if stage
                    else "Encoding annotations"
                ),
            ):
            tmp = self.vocab.encode_as_ids(["<start>"]+[token for token in ann.split()]+["<end>"])
            max_len = max(max_len, len(tmp))
            self.targets.append([v[0] for v in tmp])
            self.embeddings.append([v[1] for v in tmp])
        print(max_len)
        



    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        video_path = os.path.join(
            self.base_dir, "videos_phoenix", "videos", self.video_names[idx] + ".mp4"
        )

        video = torchvision.io.read_video(
            video_path,
            pts_unit="sec",
            output_format="TCHW",
        )[0]
        

        if self.data_hparams.sample_random:
            indices = sorted(random.sample(
                range(len(video)), len(video) * self.data_hparams.sample_percentage
            ))
        elif self.data_hparams.sample_uniform:
            indices = np.linspace(
                0,
                len(video) - 1,
                int(len(video) * self.data_hparams.sample_percentage),
            ).tolist()
        else:
            indices = list(range(len(video)))
            

        T = len(indices)
        video_tensor = video.unsqueeze(0)
        video_tensor = rearrange(
                self.transform(
                    rearrange(video_tensor[:, indices], "b t c h w -> (b t) c h w")
                ), "(b t) c h w -> b t c h w", c=3, t=T
        ).squeeze(0)
       # pdb.set_trace()
        # print(idx)
        tgt = torch.tensor(self.targets[idx], dtype=torch.long)
        tgt_embedding = torch.tensor(self.embeddings[idx], dtype=torch.float)
        #print(type(tgt))

        return video_tensor, torch.cat([tgt[1:],torch.full((max(0, 64 - tgt.size(0) + 1),), -1)]), \
            torch.cat([tgt_embedding[:-1,:],torch.full((max(0, 64 - tgt.size(0) + 1),300), 0)]), \
            torch.cat([torch.ones(tgt.size(0)-1), torch.zeros(max(0, 64 - tgt.size(0) + 1))])

    def get_metadata(self, idx):
        return {
            "signer": self.signers[idx],
            "video_name": self.video_names[idx],
            "annotation": self.annotations[idx],
            "text": self.text[idx],
        }


class Phoenix14TDatamodule(pl.LightningDataModule):

    def __init__(
        self,
        files: PhoenixFiles,
        vocabulary: PhoenixVocabulary,
        num_workers: int,
        data_hyperparameters: PhoenixDataHyperparameters,
    ):
        super(Phoenix14TDatamodule, self).__init__()
        self.metadata_paths = files
        self.vocabulary = vocabulary
        self.workers = num_workers
        self.train = None
        self.dev = None
        self.test = None
        self.data_hparams = data_hyperparameters

        assert (
            self.data_hparams.sample_random != self.data_hparams.sample_uniform
        ), "Both random and uniform sampling cannot be enabled at the same time"

    def setup(self, stage):
        self.train = Phoenix14TDataset(
            self.metadata_paths.train, self.vocabulary, self.data_hparams
        )
        self.train.setup("train")

        self.dev = Phoenix14TDataset(
            self.metadata_paths.dev, self.vocabulary, self.data_hparams
        )
        self.dev.setup("dev")

        self.test = Phoenix14TDataset(
            self.metadata_paths.test, self.vocabulary, self.data_hparams
        )
        self.test.setup("test")

    def train_dataloader(self):
        return DataLoader(
            self.train,
            shuffle=True,
            batch_size=1,
            pin_memory=True,
            num_workers=self.workers,
        )

    def val_dataloader(self):
        return DataLoader(
            self.dev,
            shuffle=False,
            batch_size=1,
            pin_memory=True,
            num_workers=self.workers,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test,
            shuffle=False,
            batch_size=1,
            pin_memory=True,
            num_workers=self.workers,
        )
        


def animate_torch_tensor(video):
    
    # Adapted from https://stackoverflow.com/a/57275596
    
    %matplotlib inline
    from matplotlib import pyplot as plt
    from matplotlib import animation
    from IPython.display import HTML

    
    video_np = video.numpy().transpose(0, 2, 3, 1)

    fig = plt.figure()
    im = plt.imshow(video_np[0,:,:,:])

    plt.close() # this is required to not display the generated image

    def init():
        im.set_data(video_np[0,:,:,:])

    def animate(i):
        im.set_data(video_np[i,:,:,:])
        return im

    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video_np.shape[0],
                                interval=1000//6)
    return HTML(anim.to_html5_video())

SyntaxError: expected 'else' after 'if' expression (2431605598.py, line 80)

In [35]:
def xxs_mvit(vocab):
    return MobileViT(
        dims=[64, 80, 96],
        conv_channels=[16, 16, 24, 24, 48, 48, 64, 64, 80, 80, 320],
        num_classes=len(vocab),
        vocabulary=vocab,
        expand_ratio=2,
        patch_size=(2, 2),
        debug=True
    )


def xs_mvit(vocab):
    return MobileViT(
        dims=[96, 120, 144],
        conv_channels=[16, 32, 48, 48, 64, 64, 80, 80, 96, 96, 384],
        num_classes=len(vocab),
        vocabulary=vocab,
        expand_ratio=4,
        patch_size=(4, 4),
    )


def s_mvit(vocab):
    return MobileViT(
        dims=[144, 192, 240],
        conv_channels=[16, 32, 64, 64, 96, 96, 128, 128, 160, 160, 640],
        num_classes=len(vocab),
        vocabulary=vocab,
        expand_ratio=4,
        patch_size=(8, 8),
    )

In [7]:
import wandb

# UNCOMMENT THIS TO LOG TO WANDB
# from kaggle_secrets import UserSecretsClient

# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("wandb_api")


wandb.login(key="aaf831dabc88d936d4e6b439b798bb4cb42814ea")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [37]:
files = PhoenixFiles(
    train="/kaggle/input/phoenix14t.pami0.train.annotations_only.gzip",
    dev="/kaggle/input/phoenix14t.pami0.dev.annotations_only.gzip",
    test="/kaggle/input/phoenix14t.pami0.test.annotations_only.gzip",
)

transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(
            (192, 256), interpolation=torchvision.transforms.InterpolationMode.BICUBIC
        ),  # from uint8 to float32
        lambda x: x / 255.0,
    ]
)

data_hparams = PhoenixDataHyperparameters(
    num_workers=3,
    batch_size=1,
    sample_random=False,
    sample_uniform=True,
    sample_percentage=1 / 3,
    transforms=transforms,
)

vocab = PhoenixVocabulary(files)

dm = Phoenix14TDatamodule(files, vocab, 1, data_hparams)

loading embeddings



Extracting tokens from phoenix14t.pami0.train.annotations_only.gzip: 100%|██████████| 7096/7096 [00:00<00:00, 80952.80it/s]

Extracting tokens from phoenix14t.pami0.dev.annotations_only.gzip: 100%|██████████| 519/519 [00:00<00:00, 62664.63it/s]

Extracting tokens from phoenix14t.pami0.test.annotations_only.gzip: 100%|██████████| 642/642 [00:00<00:00, 69279.18it/s]


In [117]:
dm = Phoenix14TDatamodule(files, vocab, 3, data_hparams)

In [118]:
dm.setup("fit")


Encoding annotations for train: 100%|██████████| 7096/7096 [00:00<00:00, 106649.49it/s]


32



Encoding annotations for dev: 100%|██████████| 519/519 [00:00<00:00, 98732.03it/s]


20



Encoding annotations for test: 100%|██████████| 642/642 [00:00<00:00, 104019.13it/s]

20


In [40]:
dm.train.targets[0][0]


(1115,
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
     

## Check out a video

In [ ]:
animate_torch_tensor(dm.train[0][0])

In [139]:
# flush cuda cache
torch.cuda.empty_cache()

import lightning.pytorch as lp

# wandb.init(project="mobilevit")

# setup training and wandb

BATCH_SIZE = 1  # we don't do padding, so batch size must be 1

# wandb_logger = lp.loggers.WandbLogger()

model = xxs_mvit(vocab)


trainer = pl.Trainer(
    max_epochs=100,
    accelerator="cpu",#cuda
    #logger=wandb_logger,
    callbacks=[
        lp.callbacks.ModelCheckpoint(
            monitor="val/loss",
            filename="best_model",
            save_top_k=1,
            mode="min",
        ),
        lp.callbacks.EarlyStopping(
            monitor="val/loss",
            patience=3,
            mode="min",
        ),
        lp.callbacks.LearningRateMonitor(logging_interval="step"),
    ],
    limit_train_batches=0.25,
    accumulate_grad_batches=32,
)

# wandb_logger.watch(model, log_graph=False)

trainer.fit(model, datamodule=dm)

wandb.finish()

INFO: GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs

Encoding annotations for train: 100%|██████████| 7096/7096 [00:00<00:00, 114867.20it/s]


32



Encoding annotations for dev: 100%|██████████| 519/519 [00:00<00:00, 71120.09it/s]


20



Encoding annotations for test: 100%|██████████| 642/642 [00:00<00:00, 73588.30it/s]
INFO: 
  | Name        | Type              | Params | Mode 
----------------------------------------------------------
0 | in_conv     | Conv2DBlock       | 464    | train
1 | mv2_blocks  | ModuleList        | 45.7 K | train
2 | mvit_blocks | ModuleList        | 1.1 M  | train
3 | final_pw    | Conv2DBlock       | 26.2 K | train
4 | decoder     | VideoDecoder      | 2.6 M  | train
5 | pool        | AdaptiveAvgPool2d | 0      | train
6 | classifier  | Linear            | 357 K  | train
7 | criterion   | CrossEntropyLoss  | 0      | train
8 | wer         | WordErrorRate     | 0      | train
----------------------------------------------------------
4.1 M     Trainable params
0         Non-trainable params
4.1 M     Total params
16.459    Total estimated model params size (MB)
344       Modules in train mode
0         Modules in eval mode


...

ipdb>  n


> /tmp/ipykernel_24/3572483869.py(649)string_from_ground_truth()
    647         for target in y:
    648             if isinstance(target, torch.Tensor):
--> 649                 target = target.tolist()
    650             if target[0] != -1: #padding
    651                 target_strings.append(self.vocabulary.decode_from_ids(target))



ipdb>  target_strings


[]


ipdb>  q


In [ ]:
xxs_kwargs = {
    "dims": [64, 80, 96],
    "conv_channels": [16, 16, 24, 24, 48, 48, 64, 64, 80, 80, 320],
    "num_classes": len(vocab),
    "vocabulary": vocab,
    "expand_ratio": 2,
    "patch_size": (2, 2),
}
get_vector
model = xxs_mvit(vocab)

In [ ]:
with torch.no_grad():
    model = model.cuda()
    x, y = dm.train[53]
    x = x.cuda().unsqueeze(0)

    logits = model(x)
    decoded = model.ctc_decode(logits)
    print(torch.tensor(decoded).min())

    print("Predicted:", vocab.decode_from_ids(decoded[0]))

In [ ]:
len(" ".join(vocab.decode_from_ids(y.tolist())))